#  <center>CANN Implementation with BrainPy</center>
   <center>王宇哲 1800011828</center>
   <center>College of Chemistry and Molecular Engineering, Peking University</center>

In [2]:
import brainpy as bp
import brainpy.math as bm
import matplotlib.pyplot as plt
%matplotlib inline

The mathematical equation of the Continuous-Attractor Neural Network (CANN) is given by:

$$\tau \frac{du(x,t)}{dt} = -u(x,t) + \rho \int dx' J(x,x') r(x',t)+I_{ext}$$

$$r(x,t) = \frac{u(x,t)^2}{1 + k \rho \int dx' u(x',t)^2}$$

$$J(x,x') = \frac{1}{\sqrt{2\pi}a}\exp(-\frac{|x-x'|^2}{2a^2})$$

$$I_{ext} = A\exp\left[-\frac{|x-z(t)|^2}{4a^2}\right]$$

In [3]:
class CANN1D(bp.NeuGroup):
    @bp.odeint
    def int_u(self, u, t, Iext):
        r1 = bp.math.square(u)
        r2 = 1.0 + self.k * bp.math.sum(r1)
        r = r1 / r2
        Irec = bp.math.dot(self.conn_mat, r)
        du = (-u + Irec + Iext) / self.tau
        return du
    
    def __init__(self, num, tau=1., k=8.1, a=0.5, A=10., J0=4.,
               z_min=-bp.math.pi, z_max=bp.math.pi, **kwargs):
        super(CANN1D, self).__init__(size=num, **kwargs)
        # parameters
        self.tau = tau  # The synaptic time constant
        self.k = k  # Degree of the rescaled inhibition
        self.a = a  # Half-width of the range of excitatory connections
        self.A = A  # Magnitude of the external input
        self.J0 = J0  # maximum connection value

        # feature space
        self.z_min = z_min
        self.z_max = z_max
        self.z_range = z_max - z_min
        self.x = bp.math.linspace(z_min, z_max, num)  # The encoded feature values
        self.rho = num / self.z_range  # The neural density
        self.dx = self.z_range / num  # The stimulus density

        # variables
        self.u = bp.math.Variable(bp.math.zeros(num))
        self.input = bp.math.Variable(bp.math.zeros(num))

        # The connection matrix
        self.conn_mat = self.make_conn(self.x)
        
    def dist(self, d):
        d = bp.math.remainder(d, self.z_range)
        d = bp.math.where(d > 0.5 * self.z_range, d - self.z_range, d)
        return d
    
    def make_conn(self, x):
        assert bp.math.ndim(x) == 1
        x_left = bp.math.reshape(x, (-1, 1))
        x_right = bp.math.repeat(x.reshape((1, -1)), len(x), axis=0)
        d = self.dist(x_left - x_right)
        Jxx = self.J0 * bp.math.exp(-0.5 * bp.math.square(d / self.a)) / \
              (bp.math.sqrt(2 * bp.math.pi) * self.a)
        return Jxx
    
    def get_stimulus_by_pos(self, pos):
        return self.A * bp.math.exp(-0.25 * bp.math.square(self.dist(self.x - pos) / 0.5))
    
    def get_stimulus_by_pos_non_gaussian(self, pos):
        return self.A  * bp.math.select([self.x>=1,self.x>=-1, self.x<-1], [0.0, 1.0, 0.0])
    
    def update(self, _t, _dt):
        self.u[:] = self.int_u(self.u, _t, self.input)
        self.input[:] = 0.

## 1. Population Coding

In [4]:
cann = CANN1D(num=512, k=0.1, monitors=['u'])

I1 = cann.get_stimulus_by_pos(0.)
Iext, duration = bp.inputs.section_input(values=[0., I1, 0.],
                                         durations=[1., 8., 8.],
                                         return_length=True)
cann.run(duration=duration, inputs=('input', Iext, 'iter'), report=0.1)

Compilation used 0.0095 s.
Start running ...
Run 10.0% used 0.003 s.
Run 20.0% used 0.007 s.
Run 30.0% used 0.017 s.
Run 40.0% used 0.027 s.
Run 50.0% used 0.032 s.
Run 60.0% used 0.036 s.
Run 70.0% used 0.042 s.
Run 80.0% used 0.048 s.
Run 90.0% used 0.053 s.
Run 100.0% used 0.058 s.
Simulation is done in 0.059 s.



0.059377431869506836

In [5]:
%%capture
fig_1 = bp.visualize.animate_1D(
  dynamical_vars=[{'ys': cann.mon.u, 'xs': cann.x, 'legend': 'u'},
                  {'ys': Iext, 'xs': cann.x, 'legend': 'Iext'}],
  frame_step=1,
  frame_delay=100,
  show=False,
  save_path='cann-encoding.gif'
)

![SegmentLocal](cann-encoding.gif "segment")

### Discussion 1: Non-gaussian Stimulus

In [37]:
cann = CANN1D(num=512, k=0.1, monitors=['u'])

I1 = cann.get_stimulus_by_pos_non_gaussian(0.)
Iext, duration = bp.inputs.section_input(values=[0., I1, 0.],
                                         durations=[1., 8., 8.],
                                         return_length=True)
cann.run(duration=duration, inputs=('input', Iext, 'iter'), report=0.1)

Compilation used 0.0010 s.
Start running ...
Run 10.0% used 0.006 s.
Run 20.0% used 0.012 s.
Run 30.0% used 0.018 s.
Run 40.0% used 0.023 s.
Run 50.0% used 0.028 s.
Run 60.0% used 0.032 s.
Run 70.0% used 0.038 s.
Run 80.0% used 0.044 s.
Run 90.0% used 0.052 s.
Run 100.0% used 0.058 s.
Simulation is done in 0.058 s.



0.057852745056152344

In [38]:
%%capture
fig_1_1 = bp.visualize.animate_1D(
  dynamical_vars=[{'ys': cann.mon.u, 'xs': cann.x, 'legend': 'u'},
                  {'ys': Iext, 'xs': cann.x, 'legend': 'Iext'}],
  frame_step=1,
  frame_delay=100,
  show=False,
  save_path='cann-encoding-non-gaussian.gif'
)

![SegmentLocal](cann-encoding-non-gaussian.gif "segment")

### Discussion 2: Parameters in CANN

Parameter: $\tau$

In [7]:
%%capture
cann = CANN1D(num=512, k=0.1, tau=10.0, monitors=['u'])

I1 = cann.get_stimulus_by_pos(0.)
Iext, duration = bp.inputs.section_input(values=[0., I1, 0.],
                                         durations=[1., 8., 8.],
                                         return_length=True)
cann.run(duration=duration, inputs=('input', Iext, 'iter'), report=0.1)

fig_1_2 = bp.visualize.animate_1D(
  dynamical_vars=[{'ys': cann.mon.u, 'xs': cann.x, 'legend': 'u'},
                  {'ys': Iext, 'xs': cann.x, 'legend': 'Iext'}],
  frame_step=1,
  frame_delay=100,
  show=False,
  save_path='cann-encoding-para-tau.gif'
)

![SegmentLocal](cann-encoding-para-tau.gif "segment")

Parameter: $k$

In [9]:
%%capture
cann = CANN1D(num=512, k=10.0, monitors=['u'])

I1 = cann.get_stimulus_by_pos(0.)
Iext, duration = bp.inputs.section_input(values=[0., I1, 0.],
                                         durations=[1., 8., 8.],
                                         return_length=True)
cann.run(duration=duration, inputs=('input', Iext, 'iter'), report=0.1)

fig_1_3 = bp.visualize.animate_1D(
  dynamical_vars=[{'ys': cann.mon.u, 'xs': cann.x, 'legend': 'u'},
                  {'ys': Iext, 'xs': cann.x, 'legend': 'Iext'}],
  frame_step=1,
  frame_delay=100,
  show=False,
  save_path='cann-encoding-para-k.gif'
)

![SegmentLocal](cann-encoding-para-k.gif "segment")

Parameter: $a$

In [45]:
%%capture
cann = CANN1D(num=512, k=0.1, a=1.0, monitors=['u'])

I1 = cann.get_stimulus_by_pos(0.)
Iext, duration = bp.inputs.section_input(values=[0., I1, 0.],
                                         durations=[1., 8., 8.],
                                         return_length=True)
cann.run(duration=duration, inputs=('input', Iext, 'iter'), report=0.1)

fig_1_4 = bp.visualize.animate_1D(
  dynamical_vars=[{'ys': cann.mon.u, 'xs': cann.x, 'legend': 'u'},
                  {'ys': Iext, 'xs': cann.x, 'legend': 'Iext'}],
  frame_step=1,
  frame_delay=100,
  show=False,
  save_path='cann-encoding-para-a.gif'
)

![SegmentLocal](cann-encoding-para-a.gif "segment")

Parameter: $J_{0}$

In [46]:
%%capture
cann = CANN1D(num=512, k=0.1, J0=1.0, monitors=['u'])

I1 = cann.get_stimulus_by_pos(0.)
Iext, duration = bp.inputs.section_input(values=[0., I1, 0.],
                                         durations=[1., 8., 8.],
                                         return_length=True)
cann.run(duration=duration, inputs=('input', Iext, 'iter'), report=0.1)

fig_1_5 = bp.visualize.animate_1D(
  dynamical_vars=[{'ys': cann.mon.u, 'xs': cann.x, 'legend': 'u'},
                  {'ys': Iext, 'xs': cann.x, 'legend': 'Iext'}],
  frame_step=1,
  frame_delay=100,
  show=False,
  save_path='cann-encoding-para-J0.gif'
)

![SegmentLocal](cann-encoding-para-J0.gif "segment")

## 2. Template Matching

In [25]:
cann = CANN1D(num=512, k=8.1, monitors=['u'])

dur1, dur2, dur3 = 10., 30., 0.
num1 = int(dur1 / bp.math.get_dt())
num2 = int(dur2 / bp.math.get_dt())
num3 = int(dur3 / bp.math.get_dt())
Iext = bp.math.zeros((num1 + num2 + num3,) + cann.size)
Iext[:num1] = cann.get_stimulus_by_pos(0.5)
Iext[num1:num1 + num2] = cann.get_stimulus_by_pos(0.)
Iext[num1:num1 + num2] += 0.1 * cann.A * bp.math.random.randn(num2, *cann.size)
cann.run(duration=dur1 + dur2 + dur3, inputs=('input', Iext, 'iter'), report=0.1)

Compilation used 0.0007 s.
Start running ...
Run 10.0% used 0.021 s.
Run 20.0% used 0.044 s.
Run 30.0% used 0.071 s.
Run 40.0% used 0.106 s.
Run 50.0% used 0.141 s.
Run 60.0% used 0.175 s.
Run 70.0% used 0.204 s.
Run 80.0% used 0.236 s.
Run 90.0% used 0.323 s.
Run 100.0% used 0.350 s.
Simulation is done in 0.351 s.



0.35123682022094727

In [26]:
%%capture
fig_2 = bp.visualize.animate_1D(
  dynamical_vars=[{'ys': cann.mon.u, 'xs': cann.x, 'legend': 'u'},
                  {'ys': Iext, 'xs': cann.x, 'legend': 'Iext'}],
  frame_step=5,
  frame_delay=50,
  show=False,
  save_path='cann-decoding.gif'
)

![SegmentLocal](cann-decoding.gif "segment")

## 3. Smooth Tracking

In [27]:
cann = CANN1D(num=512, k=8.1, monitors=['u'])

dur1, dur2, dur3 = 20., 20., 20.
num1 = int(dur1 / bp.math.get_dt())
num2 = int(dur2 / bp.math.get_dt())
num3 = int(dur3 / bp.math.get_dt())
position = bp.math.zeros(num1 + num2 + num3)
position[num1: num1 + num2] = bp.math.linspace(0., 12., num2)
position[num1 + num2:] = 12.
position = position.reshape((-1, 1))
Iext = cann.get_stimulus_by_pos(position)
cann.run(duration=dur1 + dur2 + dur3, inputs=('input', Iext, 'iter'), report=0.1)

Compilation used 0.0000 s.
Start running ...
Run 10.0% used 0.118 s.
Run 20.0% used 0.185 s.
Run 30.0% used 0.230 s.
Run 40.0% used 0.302 s.
Run 50.0% used 0.369 s.
Run 60.0% used 0.396 s.
Run 70.0% used 0.447 s.
Run 80.0% used 0.498 s.
Run 90.0% used 0.550 s.
Run 100.0% used 0.574 s.
Simulation is done in 0.574 s.



0.5744638442993164

In [28]:
%%capture
fig_3 = bp.visualize.animate_1D(
  dynamical_vars=[{'ys': cann.mon.u, 'xs': cann.x, 'legend': 'u'},
                  {'ys': Iext, 'xs': cann.x, 'legend': 'Iext'}],
  frame_step=5,
  frame_delay=50,
  show=False,
  save_path='cann-tracking.gif'
)

![SegmentLocal](cann-tracking.gif "segment")